## Extracción, Tratamiento y Carga de datos (ETL)

### Objetivo

El objetivo de este notebook es realizar un proceso de ETL del archivo JSON "australian_user_items". Como primer paso de la Ingeniería de datos, los datos se prepararán y limpiarán para su posterior análisis.

### Archivo de Datos: australian_user_items

El archivo "australian_user_items" contiene información sobre los items (o juegos) jugados por los usuarios de la plataforma STEAM (plataforma multinacional de videojuegos). 
La información que nos muestra es: user_id, user_url, items.

El detalle de los datos se encuentran en el siguiente [LINK](https://docs.google.com/spreadsheets/d/1y2FbOU9jsJbJkdmBLC7Lr3u576yENh0Z/edit?usp=sharing&ouid=100343940744383618882&rtpof=true&sd=true) con el diccionario de datos del dataset.

### 1. Carga de datos

Librerías

In [7]:
#pip install matplotlib
#pip install seaborn
#pip install pyarrow 
#pip install fastparquet


import pandas as pd
from pandas import json_normalize
import numpy as np
import json

# import warnings
# warnings.filterwarnings("ignore")

Cargamos el archivo json. El archivo se llama "australian_users_items" el cual se utilizará en adelante y se encuentra en el siguiente [link](https://drive.google.com/drive/folders/1PvMb8F0veZYFmmcZytfVBNvmfzi6Rwl9?usp=sharing)

In [11]:
# Demora en ejecutar (5 minutos aprox.)

import ast # Módulo para evaluar expresiones literales de Python (seguro si los datos son confiables)

# Abrir el archivo y cargar las líneas en una lista
with open('australian_users_items.json', 'r', encoding='utf-8') as f:
    base = f.readlines() #Se abre el archivo 'australian_users_items.json' en modo de lectura ('r') y se lee cada línea del archivo en una lista llamada base

# Evaluar cada línea como una expresión literal de Python
data = [eval(line.strip()) for line in base]
# Crear DataFrame a partir de los registros
df_User_Items= pd.DataFrame(data)

In [19]:
df_User_Items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [15]:
print(f"El DataFrame tiene {df_User_Items.shape[0]} filas y {df_User_Items.shape[1]} columnas.")


El DataFrame tiene 88310 filas y 5 columnas.


In [20]:
df=df_User_Items.copy()

### 2. Exploramos el Dataframe

In [16]:
df_User_Items.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')

In [17]:
df_User_Items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


Del dataframe **df_User_Items** se observa lo siguiente:

| Variable  | Descripción                                      |
|-----------|-------------------------------------------------------|
| user_id   | Identificador único del usuario                  |
| items_count  | Cantidad de items                         |
| steam_id   | Identificador único |
| user_url   | El url del perfil del usuario                 |
| items  |   Items del usuao, aparentemente en formato json            |

Cada 'items' de un usuario es una lista de diccionarios en formato JSON. Cada elemento de la lista contiene pares clave-valor (como "item_id", "item_name", "playtime_forever" y "playtime_2weeks") con información de los items relacionados a los usuarios. 

Exploremos a más detalle la variable 'items'.

In [25]:
# Observamos el contenido para la fila 10
df_User_Items['items'][10]


[{'item_id': '4000',
  'item_name': "Garry's Mod",
  'playtime_forever': 2644,
  'playtime_2weeks': 0},
 {'item_id': '1250',
  'item_name': 'Killing Floor',
  'playtime_forever': 30266,
  'playtime_2weeks': 0},
 {'item_id': '35420',
  'item_name': 'Killing Floor Mod: Defence Alliance 2',
  'playtime_forever': 54,
  'playtime_2weeks': 0},
 {'item_id': '6060',
  'item_name': 'STAR WARS™ Battlefront™ II',
  'playtime_forever': 86,
  'playtime_2weeks': 0},
 {'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 108,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 45,
  'playtime_2weeks': 0},
 {'item_i

In [31]:
# Demora en ejecutar (3 minutos aprox.)
# Convierte los datos JSON de la columna 'items' en un DataFrame plano excepto las columnas 'user_id','items_count','steam_id' y 'user_url'
df_User_Items = pd.json_normalize(records, record_path=['items'], meta=['user_id','items_count','steam_id','user_url'])

In [35]:
df_user_items = df_User_Items.copy()

In [36]:
df_user_items.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [37]:
df_user_items.shape

(5153209, 8)

Visualizaremos los datos de las nuevas columnas para saber qué tipo de información contienen. Para lo cual utilizaremos el método **unique()**, este método se aplica a la columna seleccionada y devuelve un array que contiene los valores únicos presentes en esa columna.

In [38]:
df_user_items.columns 

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'user_id', 'items_count', 'steam_id', 'user_url'],
      dtype='object')

In [39]:
df_user_items['item_id'].unique()

array(['10', '20', '30', ..., '354280', '433920', '485270'], dtype=object)

In [40]:
df_user_items['item_name'].unique()

array(['Counter-Strike', 'Team Fortress Classic', 'Day of Defeat', ...,
       'ChaosTower', 'Aveyond 4: Shadow Of The Mist', 'Arachnophobia'],
      dtype=object)

In [41]:
df_user_items['playtime_forever'].unique()

array([     6,      0,      7, ...,  42740, 121694,  34753], dtype=int64)

In [42]:
df_user_items['playtime_2weeks'].unique()

array([   0,  166,   67, ..., 2145, 2925, 3046], dtype=int64)

Luego de explorar los valores que contiene las nuevas columnas, se entiende lo siguiente.

Columnas del dataframe **df_user_items**:

* user_id: Identificador único del usuario
* items_count: Cantidad de items  
* steam_id: Identificador único 
* user_url: El url del perfil del usuario
    * item_id: Identificador único del 'item'
    * item_name: Nombre del 'item' 
    * playtime_forever: Cuantifica el tiempo de juego en minutos
    * playtime_2weeks: Se puede especular que cuantifica el tiempo de juego en minutos cada dos semanas.

In [43]:
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
 5   items_count       object
 6   steam_id          object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB


Ahora que ya conocemos qué información nos muestra las columnas del dataframe continuaremos con la limpieza y el preprocesamiento. El objetivo es tener lista la data para cualquier proceso analítico que se realice posteriormente.

In [44]:
# En este punto, creamos una copia del dataframe como respaldo para no ejecutar todo 
df_UserItem = df_user_items.copy()
#df_UserReviews = df_UserRev.copy()

LIMPIEZA Y PREPROCESAMIENTO DE LOS DATOS

#### 2.1 Identificar y tratar datos nulos

Primero identificamos cuántos datos tienen '', 'null' o 'None' entre sus datos:

In [48]:
total_blanco = (df_UserItem == '').sum().sum()
# Mostrar los resultados
print("Cantidad de valores blanco ('') en todo el DataFrame:", total_blanco)

total_nulos = (df_UserItem == 'null').sum().sum()
# Mostrar los resultados
print("Cantidad de valores nulos en todo el DataFrame:", total_nulos)

total_vacios = (df_UserItem == 'None').sum().sum()
# Mostrar los resultados
print("Cantidad de valores 'None' en todo el DataFrame:", total_vacios)

Cantidad de valores blanco ('') en todo el DataFrame: 0
Cantidad de valores nulos en todo el DataFrame: 0
Cantidad de valores 'None' en todo el DataFrame: 0


In [50]:
# Verifica nulos en el dataframe
df_UserItem.isnull().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
user_id             0
items_count         0
steam_id            0
user_url            0
dtype: int64

No se identificaron valores faltantes

#### 2.2 Identificar filas duplicadas

In [45]:
# Selecciona las filas duplicadas en el DataFrame
duplicados = df_UserItem[df_UserItem.duplicated(keep=False)] #keep=False significa que todas las instancias duplicadas (tanto la primera como las subsiguientes) se marcarán como duplicadas

# Cuenta el número total de filas duplicadas
num_duplicados = duplicados.shape[0]

# Muestra las filas duplicadas junto con las originales
print("Filas duplicadas:")
duplicados

Filas duplicadas:


,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
4344,4000,Garry's Mod,269,0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
4345,33910,Arma 2,162,0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
4346,33930,Arma 2: Operation Arrowhead,223,0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
4347,219540,Arma 2: Operation Arrowhead Beta (Obsolete),0,0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
4348,400,Portal,196,0,76561198156664158,59,76561198156664158,http://steamcommunity.com/profiles/76561198156...
...,...,...,...,...,...,...,...,...
4898223,213670,South Park™: The Stick of Truth™,725,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898224,221910,The Stanley Parable,53,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898225,261030,The Walking Dead: Season Two,253,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898226,273110,Counter-Strike Nexon: Zombies,0,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...


Se procede con la eliminación de las filas duplicadas, los duplicados pueden causar inconsistencia en nuestros datos, eliminarlos asegura que utilicemos datos válidos antes de la carga en el contexto de ETL.

In [51]:
# Elimina las filas duplicadas y actualiza el DataFrame
df_UserItems_without_duplicates = df_UserItem.drop_duplicates(keep='first')

# Mostrar información sobre la eliminación de duplicados
print("Número de filas antes de eliminar duplicados:", df_UserItem.shape[0])
print("Número de filas después de eliminar duplicados:", df_UserItems_without_duplicates.shape[0])


Número de filas antes de eliminar duplicados: 5153209
Número de filas después de eliminar duplicados: 5094105


Comprobamos que no haya filas duplicadas

In [52]:
# Selecciona las filas duplicadas en el DataFrame
dupli = df_UserItems_without_duplicates[df_UserItems_without_duplicates.duplicated(keep=False)] #keep=False significa que todas las instancias duplicadas (tanto la primera como las subsiguientes) se marcarán como duplicadas

# Cuenta el número total de filas duplicadas
num_duplicated = dupli.shape[0]

# Muestra las filas duplicadas junto con las originales
print("Cantidad de filas duplicadas:", num_duplicated)

Cantidad de filas duplicadas: 0


In [53]:
# Cambiamos de nombre el dataframe a 'df_UserItems'
df_UserItems = df_UserItems_without_duplicates.copy() #(5094105, 8)

In [56]:
df_UserItems.columns

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'user_id', 'items_count', 'steam_id', 'user_url'],
      dtype='object')

#### 2.3 Columnas necesarias

Una variable importante para nuestro posterior análisis son las 'Horas jugadas'. Vamos a crear la columna 'Hours_played' a partir de la columna 'playtime_forever' pasandolo a horas.

In [69]:
# Convertimos de minutos a horas, se asigna a la variable 'Hours_played'
df_UserItems['Hours_played'] = df_UserItems['playtime_forever'] / 60

# Redondea los valores a tres decimales
df_UserItems['Hours_played'] = df_UserItems['Hours_played'].round(3)

In [71]:
df_UserItems.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url,Hours_played
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.100
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.000
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.117
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.000
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.000


In [70]:
df_UserItems['Hours_played'].value_counts()

Hours_played
0.000       1847730
0.017        101586
0.033         34391
0.050         31530
0.067         29127
             ...   
1275.683          1
1077.933          1
888.800           1
733.817           1
579.217           1
Name: count, Length: 48861, dtype: int64

Se observa una gran cantidad de datos (1847730) con cerca de 0 horas de juego, para no sobrecargar la base final se eliminarán estos registros. Además, los análisis posteriores se centran en la *mayor cantidad de horas jugadas* por lo que se procederá con esta decisión de eliminar los registros con 'Hours_played' igual a cero.  

In [72]:
# Filtrar las filas donde 'Hours_played' es diferente de cero
df_UserItems = df_UserItems[df_UserItems['Hours_played'] != 0]

# Opcional: Restablecer el índice del DataFrame después de filtrar
df_UserItems = df_UserItems.reset_index(drop=True)


In [75]:
df_UserItems.shape

(3246375, 9)

#### 2.4 Evaluar columnas a eliminar

In [79]:
# Se elimina las columnas que no contribuyen a nuestro posterior análisis
df_UserItems = df_UserItems.drop(['items_count','steam_id','user_url','playtime_forever','playtime_2weeks'], axis=1)
df_UserItems.columns

Index(['item_id', 'item_name', 'user_id', 'Hours_played'], dtype='object')

In [80]:
df_UserItems

,item_id,item_name,user_id,Hours_played
0,10,Counter-Strike,76561197970982479,0.100
1,30,Day of Defeat,76561197970982479,0.117
2,300,Day of Defeat: Source,76561197970982479,78.883
3,240,Counter-Strike: Source,76561197970982479,30.883
4,3830,Psychonauts,76561197970982479,5.550
...,...,...,...,...
3246370,304930,Unturned,76561198329548331,11.283
3246371,227940,Heroes & Generals,76561198329548331,0.717
3246372,388490,One Way To Die: Steam Edition,76561198329548331,0.050
3246373,521570,You Have 10 Seconds 2,76561198329548331,0.067


### 3. Cargar

En este paso se guarda los datos transformados en un formato csv y listos para su posterior análisis o uso.

In [81]:
# Los archivos se almacenan en local 
df_UserItems.to_csv('user_items_final.csv', index=False)

Opcional

In [83]:
# Menor uso de espacio de almacenamiento en comparación con CSV, especialmente para conjuntos de datos grandes.
df_UserItems.to_parquet('user_items_final.parquet', index=False)